Step 1 - Import python libraries

In [ ]:
%pip install IPython
%pip install matplotlib
%pip install pandas
%pip install seaborn
%pip install scipy
%pip install holoviews

from IPython import get_ipython
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import holoviews as hv

pd.options.mode.chained_assignment = None

Step 2 - Load and read your data file
- pyTCR accepts a single `.csv` file that should contain all the samples.
  - The following cell attempts to detect whether you are running the notebook in a Google Colab cloud environment or in a local environment, and then loads the data at the specified path.
- The `filePath` variable in the following code cell should be changed to the location of your file. The following options are supported:
  1. A `filePath` from Google Drive (to run on a cloud environment)
  2. A `filePath` from your local computer (to run on a local environment, other cloud environments should work as expected)
- Adjust the `optional_fields` according to your data

In [ ]:
# Mount Google Drive
isInGoogle = 'google.colab' in str(get_ipython())

if isInGoogle:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# Specify the path to your data in Google Drive or locally
filePath = "../combined_data.csv" # or "/content/drive/MyDrive/combined_data.csv"

df = pd.read_csv(filePath, low_memory=False, engine="c")

optional_fields = ['hospitalized']

df.head()

Usage analysis 1 - Top n highest clonotypes 

1.   change the number in the tail () to the number that you are interested in 



In [ ]:
df1=df.sort_values(['sample', 'freq'], axis=0).groupby('sample').tail(10)
df1 = df1[['freq', 'cdr3aa', 'sample', 'hospitalized']]

df1

Usage analysis 2 - Bottom n lowest clonotypes 

1.   change the number in the head () to the number that you are interested in 



In [ ]:
df2=df.sort_values(['sample', 'freq'], axis=0).groupby('sample').head(10)
df2=df2[['freq', 'cdr3aa', 'sample', 'hospitalized']]

df2

Usage analysis 3 - Top n highest V gene (D gene, J gene)
1.   change the number in the tail () to the number that you are interested in 
2.   change 'v' to other genes that you are interested in

In [ ]:
df3 = df[['freq', 'v', 'sample','hospitalized']]
df3=df3.sort_values(['sample','freq'], axis=0).groupby('sample').tail(10)

df3

Usage analysis 4 - Bottom n lowest V gene (D gene, J gene)

1.   change the number in the head () to the number that you are interested in 
2.   change 'v' to other genes that you are interested in


In [ ]:
df4 = df[['freq', 'v', 'sample','hospitalized']]
df4 = df4.sort_values(['sample','freq'], axis=0).groupby('sample').head(10)

df4

Usage analysis 5.1 - V gene weighted usage (D gene, J gene)
1.   change 'v' to other genes that you are interested in

In [ ]:
df_frequency=df.groupby(['sample','v','hospitalized'], as_index=False)['freq'].agg({'frequency':'sum'})

df_frequency

Usage analysis 5.1.1 - Test if the dataset is normally distributed
1.  the null hypothesis here is normality
2.  if the p value is greater than 0.05, we cannot reject the null hypothesis (it is a normal distribution). If the p value is smaller than 0.05, we reject the null hypothesis (it is not a normal distribution)

In [ ]:
x = stats.normaltest(df_frequency['frequency'])
x

Usage analysis 5.1.2 - Stats test
1.  if the dataset is normally distributed, use t-test (stats.ttest_ind)
*   change the group1, group2 to the groups/samples that you are interested in
*   change 'v' to other genes that you are interested in
2.  if the dataset is not normally distributed, use Wilcoxon rank-sum test (stats.ranksums)
*   change the group1, group2 to the groups/samples that you are interested in
*   change 'v' to other genes that you are interested in

In [ ]:
def usage_stat(df):
    out = {}
    uniq_vsegs = df['v'].unique()
    for V in uniq_vsegs:
        tmp = df[df['v'] == V]
        df_group1 = tmp[tmp['hospitalized'] == True]['frequency']
        df_group2 = tmp[tmp['hospitalized'] == False]['frequency']
        stat = stats.ranksums(df_group1, df_group2)
        out[V] = stat
    return out

Stats calculation - note that the p value needs to be adjusted for multiple comparisons
* adjusted p value = reported p value / number of times of comparisons [here is len(uniq_vsegs)]

In [ ]:
usage_stat(df_frequency)

Usage analysis 5.2 - V gene weighted usage heatmap (D gene, J gene)
1.   change 'v' to other genes that you are interested in

In [ ]:
# Set aesthetics
plt.style.use(['ggplot', 'seaborn-white'])
plt.figure(figsize=(40,25))
sns.set_style("white")
sns.set_context("talk")

# Prepare the data
df_frequency['frequency'] = df_frequency['frequency'].astype(float)
result = df_frequency.pivot(index='v',columns='sample',values='frequency')

# Fill the missing value in frequency with 0
result = result.fillna(0)

# Plot the heatmap
ax = sns.heatmap(result, cmap='coolwarm', cbar_kws={'label': 'V gene frequency'})
ax.set_xlabel('Sample',fontsize=25)
ax.set_ylabel('V gene',fontsize=25)
cbar_axes = ax.figure.axes[-1]

# Change the colorbar label fontsize
ax.figure.axes[-1].yaxis.label.set_size(25)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

plt.title('V gene weighted usage', fontsize=30)
sns.despine()
plt.show()

Usage analysis 5.3 - V gene weighted usage hierarchically-clustered heatmap (D gene, J gene)
1.  change 'v' to other genes that you are interested in

In [ ]:
# Reshape the dataframe to wide form dataframe
heatmap_data = pd.pivot_table(df_frequency, values='frequency', index=['v'], columns='sample')

# Fill the missing value in frequency with 0
clustermap_data = heatmap_data.fillna(0)

# Plot the hierarchically-clustered heatmap
ax = sns.clustermap(clustermap_data, figsize=(40,25),cmap="coolwarm")
ax.fig.suptitle('V gene weighted usage') 

Usage analysis 6.1 - V gene unweighted frequency result table (D gene, J gene)
1.  change 'v' to other genes that you are interested in

In [ ]:
# Select the v and sample columns
df_unweighted_frequency = df[['v', 'sample', 'hospitalized']]

# Count the v gene
df_unweighted_frequency = df_unweighted_frequency.groupby(
    ['sample', 'v', 'hospitalized'], as_index=False)['v'].agg({'count': 'count'})

# Sum the total v gene counts in each sample
df_sum_count = df_unweighted_frequency.groupby(
    ['sample', 'hospitalized'], as_index=False)['count'].agg({'sum_count': 'sum'})

# Add the sum counts of v gene to the dataframe which contains each v gene count
df_unweighted_frequency = pd.merge(df_unweighted_frequency, df_sum_count, on=[
                                   'sample', 'hospitalized'])

# Calculate the v gene unweighted frequency
df_unweighted_frequency['unweighted_frequency'] = (
    df_unweighted_frequency['count']/df_unweighted_frequency['sum_count'])

df_unweighted_frequency = df_unweighted_frequency[['sample', 'v', 'unweighted_frequency', 'hospitalized']]

df_unweighted_frequency

Usage analysis 6.1.1 - Test if the dataset is normally distributed
1.   the null hypothesis here is normality
2.   if the p value is greater than 0.05, we cannot reject the null hypothesis (it is a normal distribution). If the p value is smaller than 0.05, we reject the null hypothesis (it is not a normal distribution)

In [ ]:
x = stats.normaltest(df_unweighted_frequency['unweighted_frequency'])
x

Usage analysis 6.1.2 - Stats test
1.   if the dataset is normally distributed, use t-test (stats.ttest_ind)
*   change the group1, group2 to the groups/samples that you are interested in
*   change 'v' to other genes that you are interested in
2.   if the dataset is not normally distributed, use Wilcoxon rank-sum test (stats.ranksums)
*   change the group1, group2 to the groups/samples that you are interested in
*   change 'v' to other genes that you are interested in

In [ ]:
def unweighted_usage_stat(df):
    out = {}
    uniq_vsegs = df['v'].unique()
    for V in uniq_vsegs:
        tmp = df[df['v'] == V]
        df_group1 = tmp[tmp['hospitalized'] == True]['unweighted_frequency']
        df_group2 = tmp[tmp['hospitalized'] == False]['unweighted_frequency']

        stat = stats.ranksums(df_group1, df_group2)
        out[V] = stat
    return out

Stats calculation - note that the p value needs to be adjusted for multiple comparisons

* adjusted p value = reported p value / number of times of comparisons [here is len(uniq_vsegs)]

In [ ]:
unweighted_usage_stat(df_unweighted_frequency)

Usage analysis 6.2 - V gene weighted usage heatmap (D gene, J gene)
1.   change 'v' to other genes that you are interested in

In [ ]:
# Set aesthetics
plt.style.use(['ggplot', 'seaborn-white'])
plt.figure(figsize=(40, 25))
sns.set_style("white")
sns.set_context("talk")

# Generate axes
df_unweighted_frequency['unweighted_frequency'] = df_unweighted_frequency['unweighted_frequency'].astype(float)
result = df_unweighted_frequency.pivot(index='v', columns='sample', values='unweighted_frequency')

# Fill the missing value in frequency with 0
result = result.fillna(0)

ax = sns.heatmap(result, cmap='coolwarm', cbar_kws={'label': 'V gene unweighted frequency'})
ax.set_xlabel('Sample', fontsize=25)
ax.set_ylabel('V gene', fontsize=25)
cbar_axes = ax.figure.axes[-1]

# Change the colorbar label fontsize
ax.figure.axes[-1].yaxis.label.set_size(25)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

plt.title('V gene unweighted usage', fontsize=30)
sns.despine()
plt.show()

Usage analysis 6.3 - V gene weighted usage hierarchically-clustered heatmap (D gene, J gene)
1.   change 'v' to other genes that you are interested in

In [ ]:
# Reshape the dataframe to wide form dataframe
clustermap_data = pd.pivot_table(
    df_unweighted_frequency, values='unweighted_frequency', index=['v'], columns='sample')

# Fill the missing value in frequency with 0
clustermap_data = clustermap_data.fillna(0)

# Plot the hierarchically-clustered heatmap
ax = sns.clustermap(clustermap_data, figsize=(40, 25), cmap="coolwarm")
ax.fig.suptitle('V gene unweighted usage')

Usage analysis 7 - V-J gene usage as Sankey plot

In [ ]:
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

sample_name = "3602BW_TCRB"

df_sankey = pd.DataFrame(columns=['source', 'target', 'value'])

df_sample = df.loc[(df['sample'] == sample_name)]
df_sample = df_sample.filter(['sample', 'freq', 'v', 'j'])

# Remove the allele from the V and J gene
df_sample['v'].replace({r"\-.*$": ''}, inplace=True, regex=True)
df_sample['j'].replace({r"\-.*$": ''}, inplace=True, regex=True)

v_names = df_sample["v"].drop_duplicates().array
v_names_size = v_names.size

j_names = df_sample["j"].drop_duplicates().array
j_names_size = j_names.size

for i in range(0, v_names_size):

    data = []
    v_name = v_names[i]

    for j in range(0, j_names_size):
        j_name = j_names[j]

        df_rows = df_sample.loc[(df_sample["v"] == v_names[i]) & (
            df_sample["j"] == j_names[j])]
        freq_sum = df_rows["freq"].sum()

        if (freq_sum > 0):
            data.append(
                {'source': v_names[i], 'target': j_names[j], 'value': freq_sum})

    df_data = pd.DataFrame(data)
    df_sankey = pd.concat([df_sankey, df_data], copy=False, ignore_index=True)

print("Unique V Genes: ", v_names_size, "Unique J Genes: ", j_names_size)

graph = hv.Sankey(df_sankey)
graph.opts(cmap='Category10', label_position='left', edge_line_width=1, edge_color="source",
           width=1024, height=768, bgcolor="snow", node_alpha=1.0, node_width=40, node_sort=True)